In [ ]:
from pprint import pprint
# importing functions and modules useful for launching simulations
from main import get_options, get_min_mean_free_path, init_particles_in_system
from main import square, thruster
from main import CollisionHandler, DSMC
from main import scipy_integrate_solve_ivp, rk4, 
from main import DataAnalyser, DataSaver

In [ ]:
debug = True

In [ ]:
# defining paths to cfg 
main = 'cfg_files/main.cfg'
system = 'cfg_files/tube.cfg' # square thruster tube
saving = 'cfg_files/saving.cfg'
simulation = 'cfg_files/simulation.cfg'

cfg_path_dict = {
    'main':main,
    'system':system,
    'saving':saving,
    'simulation':simulation
}

In [ ]:
options = get_options(cfg_path_dict)
if(debug): pprint(options)

## Building system

In [ ]:
min_mean_free_path = get_min_mean_free_path(radiuses = options['main']['particles_radius'], particles_densities = options['main']['particles_densities'])
if(debug): print('Min mean free path : {} m.'.format(min_mean_free_path))

In [ ]:
system_type = options['main']['system_type']

if(system_type=='square'):
    system = square(options[system_type], min_mean_free_path)
    zone = None
    offset = [0,0]
    
elif(system_type=='thruster'):
    system = thruster(options[system_type], min_mean_free_path)
    zone = system.get_zone()
    offset = system.get_offset()

## Initializing particles

In [ ]:
lx, ly, lz = system.get_size()
res_x, res_y = system.get_space_resolutions()
if(debug):
    print('Space size : {}'.format(system.get_size()))
    print('Resolution : {}'.format(system.get_space_resolutions()))
list_particles, vmean_exp = init_particles_in_system(**options['main'], space_size = [lx,ly], space_resolution=[res_x, res_y], zone = zone, offsets = offset, verbose = False, debug = False)
# adding particles to the grid
for particle in list_particles:
    system.add(particle)

    # TODO : do as Lui said - don't save it this way. (its okay for now)
if(system_type=='thruster'):
    system.set_sparsed_space()
    pprint("List of cases in the grid that are considered : {}".format(system.grid.sparsed_space_idx))
if(debug):
    pprint({
        'mean speed':vmean_exp,
        'first particles':[part.to_string() for part in list_particles][:5],
        'walls':system.get_walls()
    })
system.plot()


## Preping simulation

In [ ]:
# update function
import numpy as np

if(system_type=='thruster'):
    from dolfin import Point

    def f(Y,t,m,q,zone,E):
        '''
        Renvoie la dérivée de Y en y (en faisant un bilan des forces notamment) pr être entré dans RK4
        Y=[x, y, z, vx, vy, vz] ce n'est pas le Y de liste_Y
        '''
        Ex, Ey = E.split(deepcopy=True)
        vx=Y[3]
        vy=Y[4]
        vz=Y[5]
        if zone.inside(Point(Y[0],Y[1])):
            ax = (1/m) * q * Ex(Y[0], Y[1])
            ay = (1/m) * q * Ey(Y[0], Y[1])
        else :
            ax = 0  #utile si les ki sont hors du mesh,
            ay = 0
        az=0
        return np.array([vx, vy, vz, ax, ay, az])
    
    args = [system.get_zone(), system.get_E()]
    
elif(system_type=='square'):
    def f(Y,t,m,q):
        vx=Y[3]
        vy=Y[4]
        vz=Y[5]

        ax = 0 
        ay = 0
        az = 0
        return np.array([vx, vy, vz, ax, ay, az])
    
    args = []

# DSMC params
N_particles_real = [int(density*lx*ly*lz) for density in options['main']['particles_densities']] # this is the REAL number of particles
N_particles_simu = [int(nb*res_x*res_y) for nb in options['main']['particles_mean_number_per_cell']]
Ne = [i/j for i, j in zip(N_particles_real,N_particles_simu)]

# TODO : see the adaptations for the DMSC with several species of particles

mean_speed = 300
DSMC_params = {
    'vr_max' : 2*mean_speed,
    'effective_diameter':  2*options['main']['particles_radius'][0], # for now
    'Ne' : Ne[0], # this is the number of real particles one simulated particle represents.
    'cell_volume' : lx*ly*lz/(res_x*res_y), # since we are in 2D I don't really know what to add here actually... For now, I add the 3rd dimension rough size, that is l3
    'mean_particles_number_per_cell': sum(options['main']['particles_mean_number_per_cell']) # we don't account for the type
}

if(debug):
    pprint([N_particles_real, N_particles_simu, Ne, DSMC_params])
    
    
use_particles_collisions = False
use_DSMC = True

simu = options['simulation']
if(simu['scheme']=='euler_explicit'):
    integration_scheme = euler_explicit # scipy_integrate_solve_ivp , rk4 , euler_explicit
    # TODO : make a function get_scheme(str)
    
# TODO : include system instead of everything else here.
#collisionHandler = CollisionHandler(list_particles, system.get_walls(), f, eta = 0, p = 0, use_particles_collisions = use_particles_collisions, \
#        use_DSMC = use_DSMC, grid = system.get_grid(), DSMC_params = DSMC_params, integration_scheme=integration_scheme)
handler = DSMC(system = system, dsmc_params = DSMC_params, integration_scheme = integration_scheme, f = f)

## Launching Simulation

In [ ]:
if(save_test):
    params_dict = {
        'id_test' : str(id_test),
        'total_number_of_particles' : N_particles_simu,
        'path_to_data' : saving_directory+str(id_test),
        'real_particle_density' : real_particle_density,
        'effective_diameter' : effective_diameter,
        'max_speed' : max_speed,
        'mean_particles_number_per_cell' : mean_particles_number_per_cell,
        'nb_cells' : nb_cells,
        'mean_free_path' : mean_free_path,
        'mean_free_time' : mean_free_time,
        'dt' : dt,
        'MAX_INTEGRATION_STEP' : MAX_INTEGRATION_STEP,
        'nb_I' : N_particles_simu,
        'nb_I_real' : N_particles_real,
        'Ne' : Ne,
        'eta': eta,
        'loss_charge_proba' : p,
        'use_particles_collisions' : use_particles_collisions,
        'use_DSMC' : use_DSMC,
        'integration_scheme' : integration_scheme.__name__,
        'speed_init_type' : speed_init_type,
        'saving_period' : saving_period,
        'number_of_collisions' : 0,
        'mean_acceptance_rate' : 0,
        'mean_proba' : 0,
        'mean_vr_norm' : 0,
        'vr_max_init' : DSMC_params['vr_max'],
        'vr_max' : 0,
        'volume' : l1*l2*l3,
        'v_mean' : vmean_exp,
        'mass' : 
    }
    data_analyser = DataSaver(list_particles, name_test = str(id_test), saving_directory = saving_directory)
    data_analyser.save_test_params(tests_summary_file_name, params_dict, use_saving_directory = False)

In [ ]:
from tqdm import tqdm

min_mean_free_time = min_mean_free_path/mean_speed
dt = simu['dt']*min_mean_free_time
t = 0

if(save_test): # before starting simulation
        data_analyser.save_everything_to_one_csv(erase = True)
        
for k in tqdm(range(simu['number_of_steps'])): #
    #if(debug): print("\nStep {} over {}...\n".format(k+1, MAX_INTEGRATION_STEP))
    handler.step(dt, t, args)
    t+=dt
    if(save_test and (k%saving_period==0 or (k == MAX_INTEGRATION_STEP-1 and k%saving_period!=0)):
        data_analyser.save_everything_to_one_csv()
        print(collisionHandler.save_collisions_matrix(name = "test_"+str(id_test)+"_collision_matrix", iteration = k))
    
    # last saving
    number_of_collisions = collisionHandler.get_collisions_count()
    mean_acceptance_rate = collisionHandler.get_acceptance_rate()
    mean_vr_norm = collisionHandler.get_mean_vr_norm()
    vr_max_final = collisionHandler.get_vr_norm()
    mean_proba = collisionHandler.get_mean_proba()
    # saving again to the csv.
    params_dict['mean_proba']=mean_proba
    params_dict['number_of_collisions']=number_of_collisions
    params_dict['mean_acceptance_rate']=mean_acceptance_rate
    params_dict['mean_vr_norm']=mean_vr_norm
    params_dict['vr_max']=vr_max_final

    data_analyser.update_saved_params(tests_summary_file_name, params_dict, use_saving_directory = False)
# that is all